In [4]:
pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=3f82764c95e9cd811940c2bbf443a7f14f51e5490b007d437c25d70ec7376547
  Stored in directory: /Users/pandhari/Library/Caches/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


### Wikipedia Retriever

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2 , lang='en')

In [3]:
# Define your Query
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get the relevant Wikipedia documents
docs = retriever.invoke(query)

In [4]:
# Print retrieved content
for i , doc in enumerate(docs):
    print(f"\n --- Result {i+1} ---")
    print(f"Content:\n {doc.page_content}...") # truncate for display


 --- Result 1 ---
Content:
 The United States had been providing military aid and economic assistance to Pakistan for various purposes since 1948. Recently U.S. stopped military aid to Pakistan, which was about US$2 billion per year. With U.S. military assistance suspended in 2018 and civilian aid reduced to about $300 million for 2022, Pakistani authorities have turned to other countries for help.


== History ==
From 1947 to 1958, under civilian leadership, the United States provided Pakistan with modest economic aid and limited military assistance. During this period, Pakistan became a member of the South East Asian Treaty Organization (SEATO) and the Central Treaty Organization (CENTO), after a Mutual Defence Assistance Agreement signed in May 1954, which facilitated increased levels of both economic and military aid from the U.S.
In 1958, Ayub Khan led Pakistan's first military coup, becoming Chief Martial Law Administrator (CMLA) and later President until 1969. During his tenure

### Vector Store Retriever

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [6]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [14]:
# Step 2: Initialize embedding model


# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents , 
    embedding=embeddings ,
    collection_name='my_collection'
)

In [15]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs = {"k":2})

In [17]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [18]:
for i , doc in enumerate(results):
    print(f"\n --- Result {i+1} ---")
    print(doc.page_content)


 --- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

 --- Result 2 ---
LangChain helps developers build LLM applications easily.


### MMR

In [19]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [20]:
from langchain_community.vectorstores import FAISS

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs , 
    embedding=embeddings
)

In [27]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type = "mmr", # This enables MMR
    search_kwargs = {"k":3 , "lambda_mult":0.5} # k = top_results , lambda_mult = relevance-diversity balance
)

In [28]:
query = "What is Langchain?"
results = retriever.invoke(query)

In [29]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


### Multi-Query Retriever

In [30]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [31]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [32]:
# Create  FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs , embedding=embeddings)

In [33]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type = "similarity" , search_kwargs = {"k":5})

In [37]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")

In [38]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs = {"k":5}) , 
    llm = llm
)

In [39]:
# Query
query = "How to improve energy levels and maintain balance?"

In [40]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [41]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 5 ---
Photosynthesis enables plants to produce energy by converting sunlight.
******************************************************************************************************************************************************

--- Result 1 ---
Python balances readability with power, making it a popular system design language.

--- Result 2 ---
Black holes bend spacetime and store immense gravitational energy.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Photosynthesis enables plants to produce energy by converting sunligh

### ContextualCompressionRetriever

In [43]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [44]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [46]:
# Create a FAISS vector store from the documents
vectorstore = FAISS.from_documents(docs, embeddings)

In [47]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [48]:
# Set up the compressor using an LLM
compressor = LLMChainExtractor.from_llm(llm)

In [49]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [50]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [51]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The extracted relevant part is:

>The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
NO_OUTPUT.
